# Task 3: Try Unet

https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel



Start simple with provided segmentation models from pytorch
https://github.com/qubvel/segmentation_models.pytorch

also https://amaarora.github.io/2020/09/13/unet.html
and https://github.com/amaarora/amaarora.github.io/blob/master/nbs/Training.ipynb

https://www.pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/

In [1]:
from task3.utils.utils import init
from task3.utils.config import get_data_loader, get_optimizer, get_model
from task3.utils.data_utils import evaluate
from task3.utils.img_utils import show_img_batch
import importlib
import sys
import time

# PyTorch model and training necessities
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import segmentation_models_pytorch as smp
from torchmetrics import IoU
from torch.nn import BCEWithLogitsLoss

# Image datasets and image manipulation
import torchvision

# Image display
import matplotlib.pyplot as plt
import numpy as np

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

## Params

In [ ]:
config = init(config='configs/raphaela.yaml')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

## Define Model

In [ ]:
config['model'].get('smp-unet')

In [ ]:
model = smp.Unet(**config['model'].get('smp-unet')).to(device)

## Load data

In [ ]:
training_loader, validation_loader = get_data_loader(config, mode='train', get_subset=False)
# TODO fix error key
# test_loader = get_data_loader(config, mode='test', get_subset=False)

In [ ]:
batch = next(iter(training_loader))

In [ ]:
batch_valid = next(iter(validation_loader))

In [ ]:
batch['id']

In [ ]:
batch_valid['id']

## Optimizer & Loss

Some optimization algorithms such as Conjugate Gradient and LBFGS need to reevaluate the function multiple times, so you have to pass in a closure that allows them to recompute your model.

https://pytorch.org/docs/stable/optim.html

In [ ]:
# use config file
# get_model(config)
# optimizer = get_optimizer(model, config)
# criterion = get_loss(config)

In [ ]:
#optimizer = config['training']['optimizer']
#lr = config['training']['lr']
#momentum = config['training']['momentum']
#loss = config['training']['loss']
#num_classes = config['training'].get('get_classes', 2)
#
#if optimizer == 'Adam':
#    
#elif optimizer == 'SGD':
#    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#else:
#    raise Exception('Optimizer not supported, choose among Adam and SGD.')
#
##if loss == 'iou':
#    criterion = IoU(num_classes=num_classes)
#
#elif loss == 'bcewithlogits':
#    assert config['model']['smp-unet']['activation'] == None, f'Last layer of the Unet model should not be sigmoid \
#    if you are using {loss}.'
#    # Sigmoid + BCELoss, numerically more stable 
#    criterion = BCEWithLogitsLoss(pos_weight=None) 
#

optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss = smp.utils.losses.DiceLoss()    
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

## Show images on Tensorboard

In [ ]:
# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

# Extract a batch of 4 images
dataiter = iter(training_loader)
batch = dataiter.next()
images, labels = batch['frame_cropped'], batch['label_cropped']

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)


# Default log_dir argument is 'runs' - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
writer = SummaryWriter('runs/mv_experiment_1')

# Write image data to TensorBoard log dir
writer.add_image('Batch Images MV', img_grid)
writer.flush()

# To view, start TensorBoard on the command line with:
#   tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

## Training loop

In [ ]:
print(len(training_loader))
num_epochs = config['training']['epochs']
data_iter = iter(training_loader)

print("[INFO] training the network...")
startTime = time.time()

for epoch in range(num_epochs):  # loop over the dataset multiple times, start with one single epoch
    print(f'epoch {epoch} / {num_epochs}')
    running_loss = 0.0
    
    # TODO model.train() # dropout, batchnorm behave differently if train vs eval
        
    for i, batch in enumerate(training_loader):
        # basic training loop
        
        # conv expectes (n_samples, channels, height, width) # e.g., (1000, 1, 224, 224)
        # Passing grayscale images in their usual format (224, 224) won't work.
        # Need to add one channel dimension
        # https://stackoverflow.com/questions/57237381/runtimeerror-expected-4-dimensional-input-for-4-dimensional-weight-32-3-3-but
        inputs, labels = batch['frame_cropped'], batch['label_cropped'] 
        optimizer.zero_grad(set_to_none=False)
        
        # does model have flatten layer?
        # Flatten images into a xxx long vector
        #inputs = inputs.view(inputs.shape[0], -1)
        
        print(inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Every 1000 mini-batches...
            print('Batch {}'.format(i + 1))
            # Check against the validation set
            running_vloss = 0.0

            model.train(False) # Don't need to track gradents for validation
            for j, vdata in enumerate(validation_loader, 0):
                vinputs, vlabels = vdata
                voutputs = model(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item()
            model.train(True) # Turn gradients back on for training

            avg_loss = running_loss / 100
            avg_vloss = running_vloss / len(validation_loader)

            # Log the running loss averaged per batch
            writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_loss, 'Validation' : avg_vloss },
                            epoch * len(training_loader) + i)

            running_loss = 0.0
print('Finished Training')

writer.flush()

In [ ]:
model

In [ ]:


from engine import train_one_epoch, evaluate
import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [ ]:
class Block(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, output_channels, 3)
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(output_channels, output_channels, 3)
    
    def forward(self, x):
        return self.conv2(self.relu(self.conv1(x)))


class Encoder(nn.Module):
    def __init__(self, chs=(3,64,128,256,512,1024)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    
    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs


class Decoder(nn.Module):
    def __init__(self, chs=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.chs         = chs
        self.upconvs    = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)]) 
        
    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x)
            enc_ftrs = self.crop(encoder_features[i], x)
            x        = torch.cat([x, enc_ftrs], dim=1)
            x        = self.dec_blocks[i](x)
        return x
    
    def crop(self, enc_ftrs, x):
        _, _, H, W = x.shape
        enc_ftrs   = torchvision.transforms.CenterCrop([H, W])(enc_ftrs)
        return enc_ftrs


class UNet(nn.Module):
    def __init__(self, enc_chs=(3,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=1, retain_dim=False, out_sz=(572,572)):
        super().__init__()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = nn.Conv2d(dec_chs[-1], num_class, 1)
        self.retain_dim  = retain_dim

    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        if self.retain_dim:
            out = F.interpolate(out, out_sz)
        return out

train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=12)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=4)